In [96]:
import simtk.openmm.app
import simtk.openmm
import simtk.unit as unit
import configparser
import numpy as np
import itertools
import scipy.spatial.distance as sdist
import os
import pdbfixer
import pandas
import subprocess
import nose

In [97]:
def parseConfigTable(config_section):
    """ Parses a section of the configuration file as a table.
        This function is used to parse the openna.conf file"""

    def readData(config_section, a):
        """Filters comments and returns values as a list"""
        temp = config_section.get(a).split('#')[0].split()
        l = []
        for val in temp:
            val = val.strip()
            try:
                x = int(val)
                l += [x]
            except ValueError:
                try:
                    y = float(val)
                    l += [y]
                except ValueError:
                    l += [val]
        return l

    data = []
    for a in config_section:
        if a == 'name':
            columns = readData(config_section, a)
            print(columns)
        elif len(a) > 3 and a[:3] == 'row':
            data += [readData(config_section, a)]
        else:
            print(f'Unexpected row {readData(config_section, a)}')
    return pandas.DataFrame(data, columns=columns)

In [98]:
class DNA(object):
    """ A Coarse Grained DNA object."""
    def __init__(self, periodic=True):
        """Initializes an DNA object"""
        self.periodic = periodic
        
    def parseConfigurationFile(self, configuration_file=f'./openna.conf'):
        """Reads the configuration file for the forcefield. The default configuration file is 3SPN2.conf
        and it contains most of the parameters used in the simulation."""
        self.configuration_file = configuration_file
        config = configparser.ConfigParser()
        config.read(configuration_file)

        # Parse all sections of the configuration file
        self.config = {}
        for c in config.sections():
            self.config.update({c: parseConfigTable(config[c])})

        # Assign main sections to variables
        self.particle_definition = self.config['Particles']

In [99]:
TEST = DNA()

In [100]:
TEST.parseConfigurationFile()

['nucleic', 'name', 'epsilon', 'radius', 'mass', 'charge']
['nucleic', 'i', 'j', 's1', 'r0', 'Kb2', 'Kb3', 'Kb4']


In [101]:
TEST.

SyntaxError: invalid syntax (<ipython-input-101-58306b89e974>, line 1)

In [102]:
a = DNA()

In [103]:
a.parseConfigurationFile()

['nucleic', 'name', 'epsilon', 'radius', 'mass', 'charge']
['nucleic', 'i', 'j', 's1', 'r0', 'Kb2', 'Kb3', 'Kb4']


In [104]:
print(a.particle_definition)

nucleic name   epsilon  radius      mass  charge
0      RNA    P  0.239006     4.5   94.9696    -0.6
1      RNA    S  0.239006     6.2   99.1094     0.0
2      RNA    A  0.239006     5.4  134.1220     0.0
3      RNA    U  0.239006     7.1  109.0620     0.0
4      RNA    G  0.239006     4.9  150.1214     0.0
5      RNA    C  0.239006     6.4  110.0964     0.0
6      DNA    P  0.239006     4.5   94.9696    -0.6
7      DNA    S  0.239006     6.2   83.1104     0.0
8      DNA    A  0.239006     5.4  134.1220     0.0
9      DNA    T  0.239006     7.1  125.1078     0.0
10     DNA    G  0.239006     4.9  150.1214     0.0
11     DNA    C  0.239006     6.4  110.0964     0.0


In [105]:
a.configuration_file

'./openna.conf'

In [106]:
a.config

{'Particles':    nucleic name   epsilon  radius      mass  charge
 0      RNA    P  0.239006     4.5   94.9696    -0.6
 1      RNA    S  0.239006     6.2   99.1094     0.0
 2      RNA    A  0.239006     5.4  134.1220     0.0
 3      RNA    U  0.239006     7.1  109.0620     0.0
 4      RNA    G  0.239006     4.9  150.1214     0.0
 5      RNA    C  0.239006     6.4  110.0964     0.0
 6      DNA    P  0.239006     4.5   94.9696    -0.6
 7      DNA    S  0.239006     6.2   83.1104     0.0
 8      DNA    A  0.239006     5.4  134.1220     0.0
 9      DNA    T  0.239006     7.1  125.1078     0.0
 10     DNA    G  0.239006     4.9  150.1214     0.0
 11     DNA    C  0.239006     6.4  110.0964     0.0,
 'Bonds':   nucleic  i  j  s1     r0       Kb2  Kb3        Kb4
 0     RNA  P  S   0  4.157  0.143403    0  14.340344
 1     RNA  S  P   1  3.780  0.143403    0  14.340344
 2     RNA  S  A   0  4.697  0.143403    0  14.340344
 3     RNA  S  U   0  4.220  0.143403    0  14.340344
 4     RNA  S  G  

In [107]:
def parsePDB(pdb_file):
    """ Transforms the pdb file into a pandas table for easy access and data editing"""

    def pdb_line(line):
        return dict(recname=str(line[0:6]).strip(),  # record name
                    serial=int(line[6:11]),          # atom serial number
                    name=str(line[12:16]).strip(),   # atom name
                    altLoc=str(line[16:17]),         # alternate location indicator
                    resname=str(line[17:20]).strip(),
                    chainID=str(line[21:22]),
                    resSeq=int(line[22:26]),         # residue sequence number
                    iCode=str(line[26:27]),          # code for insertion of residues
                    x=float(line[30:38]),
                    y=float(line[38:46]),
                    z=float(line[46:54]),
                    occupancy=0.0 if line[54:60].strip() == '' else float(line[54:60]),
                    tempFactor=0.0 if line[60:66].strip() == '' else float(line[60:66]),
                    element=str(line[76:78]),        # element symbol, right-justified
                    charge=str(line[78:80]))         # charge on the atom, right-justified

    with open(pdb_file, 'r') as pdb:
        lines = []
        for line in pdb:
            if len(line) > 6 and line[:6] in ['ATOM  ', 'HETATM']:
                lines += [pdb_line(line)]
    pdb_atoms = pandas.DataFrame(lines)
    pdb_atoms = pdb_atoms[['recname', 'serial', 'name', 'altLoc',
                           'resname', 'chainID', 'resSeq', 'iCode',
                           'x', 'y', 'z', 'occupancy', 'tempFactor',
                           'element', 'charge']]
    return pdb_atoms

In [108]:
b = parsePDB('fiber-ssRNA.pdb')

In [109]:
print(b)

recname  serial name altLoc resname chainID  resSeq iCode      x      y  \
0      ATOM       1    P              A       A       1        3.063  8.025   
1      ATOM       2  OP1              A       A       1        3.223  8.856   
2      ATOM       3  OP2              A       A       1        1.891  7.121   
3      ATOM       4  O5'              A       A       1        4.396  7.181   
4      ATOM       5  C5'              A       A       1        5.621  7.881   
..      ...     ...  ...    ...     ...     ...     ...   ...    ...    ...   
227    ATOM     228   N3              C       A      11        2.848  4.363   
228    ATOM     229   C4              C       A      11        1.578  4.062   
229    ATOM     230   N4              C       A      11        1.152  2.830   
230    ATOM     231   C5              C       A      11        0.692  5.036   
231    ATOM     232   C6              C       A      11        1.197  6.278   

          z  occupancy  tempFactor element charge  
0  

In [110]:
b.columns

Index(['recname', 'serial', 'name', 'altLoc', 'resname', 'chainID', 'resSeq',
       'iCode', 'x', 'y', 'z', 'occupancy', 'tempFactor', 'element', 'charge'],
      dtype='object')

In [111]:
b.set_index('name')

,recname,serial,altLoc,resname,chainID,resSeq,iCode,x,y,z,occupancy,tempFactor,element,charge
name,,,,,,,,,,,,,,
P,ATOM,1,,A,A,1,,3.063,8.025,-4.135,1.0,1.0,P,
OP1,ATOM,2,,A,A,1,,3.223,8.856,-5.350,1.0,1.0,O,
OP2,ATOM,3,,A,A,1,,1.891,7.121,-4.118,1.0,1.0,O,
O5',ATOM,4,,A,A,1,,4.396,7.181,-3.881,1.0,1.0,O,
C5',ATOM,5,,A,A,1,,5.621,7.881,-3.587,1.0,1.0,C,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
N3,ATOM,228,,C,A,11,,2.848,4.363,-26.363,1.0,1.0,N,
C4,ATOM,229,,C,A,11,,1.578,4.062,-26.668,1.0,1.0,C,
N4,ATOM,230,,C,A,11,,1.152,2.830,-26.436,1.0,1.0,N,


In [112]:
b['name'].unique()

array(['P', 'OP1', 'OP2', "O5'", "C5'", "C4'", "O4'", "C3'", "O3'", "C2'",
       "O2'", "C1'", 'N9', 'C8', 'N7', 'C5', 'C6', 'N6', 'N1', 'C2', 'N3',
       'C4', 'O6', 'N2', 'O2', 'O4', 'N4'], dtype=object)

In [113]:
def CoarseGrain(pdb_table):
    """ Selects RNA atoms from a pdb table and returns a table containing only the coarse-grained atoms for 3SPN2"""
    masses = {"H": 1.00794, "C": 12.0107, "N": 14.0067, "O": 15.9994, "P": 30.973762, }
    CG = {"O5\'": 'P', "C5\'": 'S', "C4\'": 'S', "O4\'": 'S', "C3\'": 'S', "O3\'": 'P', "O2\'": 'S',
          "C2\'": 'S', "C1\'": 'S', "O5*": 'P', "C5*": 'S', "C4*": 'S', "O4*": 'S',
          "C3*": 'S', "O3*": 'P', "C2*": 'S', "C1*": 'S', "N1": 'B', "C2": 'B', "O2": 'B',
          "N2": 'B', "N3": 'B', "C4": 'B', "N4": 'B', "C5": 'B', "C6": 'B', "N9": 'B',
          "C8": 'B', "O6": 'B', "N7": 'B', "N6": 'B', "O4": 'B', "C7": 'B', "P": 'P',
          "OP1": 'P', "OP2": 'P', "O1P": 'P', "O2P": 'P', "OP3": 'P', "HO5'": 'P',
          "H5'": 'S', "H5''": 'S', "H4'": 'S', "H3'": 'S', "H2'": 'S', "H2''": 'S',
          "H1'": 'S', "H8": 'B', "H61": 'B', "H62": 'B', 'H2': 'B', 'H1': 'B', 'H21': 'B',
          'H22': 'B', 'H3': 'B', 'H71': 'B', 'H72': 'B', 'H73': 'B', 'H6': 'B', 'H41': 'B',
          'H42': 'B', 'H5': 'B', "HO3'": 'P'}
    cols = ['recname', 'serial', 'name', 'altLoc',
            'resname', 'chainID', 'resSeq', 'iCode',
            'x', 'y', 'z', 'occupancy', 'tempFactor',
            'element', 'charge', 'type']
    temp = pdb_table.copy()

    # Select RNA residues
    temp = temp[temp['resname'].isin(['A', 'U', 'G', 'C'])]

    # Group the atoms by sugar, phosphate or base
    temp['group'] = temp['name'].replace(CG) # https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.replace.html
    temp = temp[temp['group'].isin(['P', 'S', 'B'])] # select based on whether the value is in one column

    # Move the O3' to the next residue
    for c in temp['chainID'].unique():
        sel = temp.loc[(temp['name'] == "O3\'") & (temp['chainID'] == c), "resSeq"]
        temp.loc[(temp['name'] == "O3\'") & (temp['chainID'] == c), "resSeq"] = list(sel)[1:] + [-1]
        sel = temp.loc[(temp['name'] == "O3\'") & (temp['chainID'] == c), "resname"]
        temp.loc[(temp['name'] == "O3\'") & (temp['chainID'] == c), "resname"] = list(sel)[1:] + ["remove"]
    # temp = temp[temp['resSeq'] > 0]
    temp = temp[temp['resname'] != 'remove']

    # Calculate center of mass
    temp['element'] = temp['element'].str.strip()
    temp['mass'] = temp.element.replace(masses).astype(float)
    temp[['x', 'y', 'z']] = (temp[['x', 'y', 'z']].T * temp['mass']).T[['x', 'y', 'z']]
    temp = temp[temp['element'] != 'H']  # Exclude hydrogens
    Coarse = temp.groupby(['chainID', 'resSeq', 'resname', 'group']).sum().reset_index()
    Coarse[['x', 'y', 'z']] = (Coarse[['x', 'y', 'z']].T / Coarse['mass']).T[['x', 'y', 'z']]

    # Set pdb columns
    Coarse['recname'] = 'ATOM'
    Coarse['name'] = Coarse['group']
    Coarse['altLoc'] = ''
    Coarse['iCode'] = ''
    Coarse['charge'] = ''
    # Change name of base to real base
    mask = (Coarse.name == 'B')
    Coarse.loc[mask, 'name'] = Coarse[mask].resname.str[-1]  # takes last letter from the residue name
    Coarse['type'] = Coarse['name']
    # Set element (depends on base)
    Coarse['element'] = Coarse['name'].replace({'P': 'P', 'S': 'H', 'A': 'N', 'T': 'S', 'G': 'C', 'C': 'O'})
    # Remove P from the beggining
    drop_list = []
    for chain in Coarse.chainID.unique():
        sel = Coarse[Coarse.chainID == chain]
        drop_list += list(sel[(sel.resSeq == sel.resSeq.min()) & sel['name'].isin(['P'])].index)
    Coarse = Coarse.drop(drop_list)
    # Renumber
    Coarse.index = range(len(Coarse))
    Coarse['serial'] = Coarse.index
    return Coarse[cols]

In [114]:
coarse = CoarseGrain(b)

In [115]:
masses = {"H": 1.00794, "C": 12.0107, "N": 14.0067, "O": 15.9994, "P": 30.973762, }
CG = {"O5\'": 'P', "C5\'": 'S', "C4\'": 'S', "O4\'": 'S', "C3\'": 'S', "O3\'": 'P', "O2\'": 'S',
      "C2\'": 'S', "C1\'": 'S', "O5*": 'P', "C5*": 'S', "C4*": 'S', "O4*": 'S',
      "C3*": 'S', "O3*": 'P', "C2*": 'S', "C1*": 'S', "N1": 'B', "C2": 'B', "O2": 'B',
      "N2": 'B', "N3": 'B', "C4": 'B', "N4": 'B', "C5": 'B', "C6": 'B', "N9": 'B',
      "C8": 'B', "O6": 'B', "N7": 'B', "N6": 'B', "O4": 'B', "C7": 'B', "P": 'P',
      "OP1": 'P', "OP2": 'P', "O1P": 'P', "O2P": 'P', "OP3": 'P', "HO5'": 'P',
      "H5'": 'S', "H5''": 'S', "H4'": 'S', "H3'": 'S', "H2'": 'S', "H2''": 'S',
      "H1'": 'S', "H8": 'B', "H61": 'B', "H62": 'B', 'H2': 'B', 'H1': 'B', 'H21': 'B',
      'H22': 'B', 'H3': 'B', 'H71': 'B', 'H72': 'B', 'H73': 'B', 'H6': 'B', 'H41': 'B',
      'H42': 'B', 'H5': 'B', "HO3'": 'P'}
cols = ['recname', 'serial', 'name', 'altLoc',
        'resname', 'chainID', 'resSeq', 'iCode',
        'x', 'y', 'z', 'occupancy', 'tempFactor',
        'element', 'charge', 'type']

In [116]:
temp = b.copy()

In [117]:
temp[temp['resname'].isin(['A', 'U', 'G', 'C'])]

,recname,serial,name,altLoc,resname,chainID,resSeq,iCode,x,y,z,occupancy,tempFactor,element,charge
0,ATOM,1,P,,A,A,1,,3.063,8.025,-4.135,1.0,1.0,P,
1,ATOM,2,OP1,,A,A,1,,3.223,8.856,-5.350,1.0,1.0,O,
2,ATOM,3,OP2,,A,A,1,,1.891,7.121,-4.118,1.0,1.0,O,
3,ATOM,4,O5',,A,A,1,,4.396,7.181,-3.881,1.0,1.0,O,
4,ATOM,5,C5',,A,A,1,,5.621,7.881,-3.587,1.0,1.0,C,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,ATOM,228,N3,,C,A,11,,2.848,4.363,-26.363,1.0,1.0,N,
228,ATOM,229,C4,,C,A,11,,1.578,4.062,-26.668,1.0,1.0,C,
229,ATOM,230,N4,,C,A,11,,1.152,2.830,-26.436,1.0,1.0,N,
230,ATOM,231,C5,,C,A,11,,0.692,5.036,-27.231,1.0,1.0,C,


In [118]:
temp['group'] = temp['name'].replace(CG)

In [119]:
temp

,recname,serial,name,altLoc,resname,chainID,resSeq,iCode,x,y,z,occupancy,tempFactor,element,charge,group
0,ATOM,1,P,,A,A,1,,3.063,8.025,-4.135,1.0,1.0,P,,P
1,ATOM,2,OP1,,A,A,1,,3.223,8.856,-5.350,1.0,1.0,O,,P
2,ATOM,3,OP2,,A,A,1,,1.891,7.121,-4.118,1.0,1.0,O,,P
3,ATOM,4,O5',,A,A,1,,4.396,7.181,-3.881,1.0,1.0,O,,P
4,ATOM,5,C5',,A,A,1,,5.621,7.881,-3.587,1.0,1.0,C,,S
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
227,ATOM,228,N3,,C,A,11,,2.848,4.363,-26.363,1.0,1.0,N,,B
228,ATOM,229,C4,,C,A,11,,1.578,4.062,-26.668,1.0,1.0,C,,B
229,ATOM,230,N4,,C,A,11,,1.152,2.830,-26.436,1.0,1.0,N,,B
230,ATOM,231,C5,,C,A,11,,0.692,5.036,-27.231,1.0,1.0,C,,B


In [120]:
for c in temp['chainID'].unique():
    sel = temp.loc[(temp['name'] == "O3\'") & (temp['chainID'] == c), "resSeq"]
    print(sel)
    temp.loc[(temp['name'] == "O3\'") & (temp['chainID'] == c), "resSeq"] = list(sel)[1:] + [-1]
    print(temp.loc[temp['name']=="O3\'"])

8       1
30      2
52      3
74      4
97      5
120     6
140     7
160     8
180     9
200    10
220    11
Name: resSeq, dtype: int64
    recname  serial name altLoc resname chainID  resSeq iCode      x      y  \
8      ATOM       9  O3'              A       A       2        7.397  5.908   
30     ATOM      31  O3'              A       A       3        9.417  0.975   
52     ATOM      53  O3'              A       A       4        8.451 -4.266   
74     ATOM      75  O3'              G       A       5        4.807 -8.157   
97     ATOM      98  O3'              G       A       6       -0.362 -9.461   
120    ATOM     121  O3'              U       A       7       -5.415 -7.766   
140    ATOM     141  O3'              U       A       8       -8.752 -3.610   
160    ATOM     161  O3'              U       A       9       -9.315  1.690   
180    ATOM     181  O3'              C       A      10       -6.926  6.456   
200    ATOM     201  O3'              C       A      11       -2.340  9.1

ValueError: cannot set using a multi-index selection indexer with a different length than the value